In [1]:
import pickle
import subprocess
train_path = '/mnt/10tb/home/shevtsov/diff_gan/ProteinGAN/data/big_diff_data_train/128-254.fasta'

In [4]:
subprocess.call(f"mmseqs createdb {train_path} DB", shell=True)

createdb /mnt/10tb/home/shevtsov/diff_gan/ProteinGAN/data/big_diff_data_train/128-254.fasta DB 

MMseqs Version:       	14.7e284
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[===================================================================================================	1 Mio. sequences processed
===================================================================================================	2 Mio. sequences processed
===================================================================================================	3 Mio. sequences processed
===================================================================================================	4 Mio. sequences processed
===================================================================================================	5 Mio. sequences processed
===================================

0

In [5]:
subprocess.call(f"mmseqs cluster DB DB_clu tmp --min-seq-id 0.7", shell=True)

Create directory tmp
cluster DB DB_clu tmp --min-seq-id 0.7 

MMseqs Version:                     	14.7e284
Substitution matrix                 	aa:blosum62.out,nucl:nucleotide.out
Seed substitution matrix            	aa:VTML80.out,nucl:nucleotide.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	seq:2147483647,prof:2147483647
Alphabet size                       	aa:21,nucl:5
Max sequence length                 	65535
Max results per query               	20
Split database                      	0
Split mode                          	2
Split memory limit                  	0
Coverage threshold                  	0.8
Coverage mode                       	0
Compositional bias                  	1
Compositional bias                  	1
Diagonal scoring                    	true
Exact k-mer matching                	0
Mask residues                       	1
Mask residues probability           	0.9
Mask lower case residues         

kmermatcher DB tmp/18308562191614635002/linclust/15300549753702370241/pref --sub-mat 'aa:blosum62.out,nucl:nucleotide.out' --alph-size aa:13,nucl:5 --min-seq-id 0.7 --kmer-per-seq 21 --spaced-kmer-mode 1 --kmer-per-seq-scale aa:0.000,nucl:0.200 --adjust-kmer-len 0 --mask 0 --mask-prob 0.9 --mask-lower-case 0 --cov-mode 0 -k 0 -c 0.8 --max-seq-len 65535 --hash-shift 67 --split-memory-limit 0 --include-only-extendable 0 --ignore-multi-kmer 0 --threads 16 --compressed 0 -v 3 

kmermatcher DB tmp/18308562191614635002/linclust/15300549753702370241/pref --sub-mat 'aa:blosum62.out,nucl:nucleotide.out' --alph-size aa:13,nucl:5 --min-seq-id 0.7 --kmer-per-seq 21 --spaced-kmer-mode 1 --kmer-per-seq-scale aa:0.000,nucl:0.200 --adjust-kmer-len 0 --mask 0 --mask-prob 0.9 --mask-lower-case 0 --cov-mode 0 -k 0 -c 0.8 --max-seq-len 65535 --hash-shift 67 --split-memory-limit 0 --include-only-extendable 0 --ignore-multi-kmer 0 --threads 16 --compressed 0 -v 3 

Database size: 18466614 type: Aminoacid
Re

0

In [6]:
subprocess.call(f"mmseqs createtsv DB DB DB_clu DB_clu.tsv", shell=True)

createtsv DB DB DB_clu DB_clu.tsv 

MMseqs Version:                 	14.7e284
First sequence as representative	false
Target column                   	1
Add full header                 	false
Sequence source                 	0
Database output                 	false
Threads                         	16
Compressed                      	0
Verbosity                       	3

Time for merging to DB_clu.tsv: 0h 0m 7s 685ms
Time for processing: 0h 0m 12s 148ms


0

Берем для val_set 20% из тех кластеров у которых 3 или меньше последовательностей. Как и делали в статье

In [1]:
import numpy as np
import pandas as pd
SEED =42
np.random.seed(SEED)

clutser_df = pd.read_table('DB_clu.tsv', header = None)

clusters_count = pd.DataFrame(clutser_df[0].value_counts())
less_3_count_clusters_list = clusters_count[clusters_count[0] <= 3].index.to_numpy()

val_clusters = np.random.choice(less_3_count_clusters_list, int(len(less_3_count_clusters_list)*0.01), replace = False)
val_cluster_df = clutser_df[clutser_df[0].isin(val_clusters)]

In [2]:
split_df = pd.DataFrame({'seq_id':clutser_df[1]})
split_df['fold'] = None

# split_df['fold'] = split_df['fold'].apply(lambda x: 'val' if x in val_ids else 'train')
split_df.loc[split_df.seq_id.isin(val_cluster_df[1]), 'fold'] = 'val'
split_df.loc[split_df.fold != 'val', 'fold'] = 'train'

In [3]:
split_df[split_df.fold == 'val'].shape, split_df[split_df.fold == 'train'].shape

((182475, 2), (18284139, 2))

In [4]:
split_df[split_df.fold == 'val']['seq_id'].to_csv('valid_ids.txt', index = False, header=False)
split_df[split_df.fold == 'train']['seq_id'].to_csv('train_ids.txt', index = False, header=False)

In [5]:
import os
os.chdir('../')

In [6]:
!awk '{ if ((NR>1)&&($0~/^>/)) { printf("\n%s", $0); } else if (NR==1) { printf("%s", $0); } else { printf("\t%s", $0); } }' 128-254.fasta | grep -Ff mmseq/valid_ids.txt - | tr "\t" "\n" > valid.fasta
!awk '{ if ((NR>1)&&($0~/^>/)) { printf("\n%s", $0); } else if (NR==1) { printf("%s", $0); } else { printf("\t%s", $0); } }' 128-254.fasta | grep -vFf mmseq/valid_ids.txt - | tr "\t" "\n" > train.fasta

In [7]:
!wc -l train_try.fasta valid_try.fasta

  36568270 train_try.fasta
    364958 valid_try.fasta
  36933228 total


In [8]:
split_df[split_df.fold == 'val'].shape[0]*2, split_df[split_df.fold == 'train'].shape[0]*2

(364950, 36568278)